In [1]:
import pickle
import re
import string
import nltk
import pandas as pd
import spacy
import re
import string
from collections import Counter
from sklearn.feature_extraction import text 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
spacy.prefer_gpu()
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

In [5]:
# Read in the document-term matrix

data_cv = pd.read_pickle('dtm_cv.pickle')
data_cv = data_cv.transpose()
data_cv

,wsb,science,ama,askreddit,worldnews,funny,dankmemes,memes,nosleep,psychology,politics,cscareerquestions,writingprompts
aa,0,10,0,9,2,0,0,0,1,2,2,0,0
aaa,5,0,0,3,0,0,0,0,0,0,2,0,0
aaaaa,0,0,0,0,0,0,0,1,0,0,0,0,0
aaaaaaaa,0,0,0,1,0,0,0,0,0,0,0,0,0
aaaaaaaaaaaa,0,0,0,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
𝙄𝙏,0,0,0,1,0,0,0,0,0,0,0,0,0
𝙏𝙃𝙀,0,0,0,2,0,0,0,0,0,0,0,0,0
𝙔𝙀𝙎,0,0,0,1,0,0,0,0,0,0,0,0,0
𝚃𝚑𝚒𝚜,0,0,0,0,0,0,0,1,0,0,0,0,0


In [8]:
# Find the top 30 words in each subreddit
top_dict_cv = {}
for c in data_cv.columns:
    top = data_cv[c].sort_values(ascending=False).head(30)
    top_dict_cv[c]= list(zip(top.index, top.values))

top_dict_cv

{'wsb': [('just', 2801),
  ('im', 2434),
  ('like', 2322),
  ('calls', 2270),
  ('dont', 1652),
  ('buy', 1650),
  ('tsla', 1555),
  ('today', 1420),
  ('going', 1381),
  ('money', 1320),
  ('day', 1286),
  ('puts', 1234),
  ('market', 1220),
  ('corn', 1210),
  ('good', 1194),
  ('bought', 1163),
  ('lol', 1160),
  ('spy', 1129),
  ('fuck', 1127),
  ('think', 1126),
  ('sell', 1126),
  ('shit', 1096),
  ('stock', 1082),
  ('time', 1071),
  ('got', 1014),
  ('people', 954),
  ('fucking', 925),
  ('right', 918),
  ('tesla', 883),
  ('thats', 863)],
 'science': [('people', 11366),
  ('just', 8939),
  ('like', 8171),
  ('dont', 7114),
  ('im', 5866),
  ('think', 4672),
  ('know', 3985),
  ('thats', 3966),
  ('time', 3796),
  ('really', 3451),
  ('make', 3370),
  ('way', 3317),
  ('good', 3059),
  ('need', 2976),
  ('years', 2861),
  ('work', 2770),
  ('doesnt', 2657),
  ('youre', 2596),
  ('going', 2591),
  ('things', 2454),
  ('lot', 2431),
  ('use', 2414),
  ('isnt', 2282),
  ('does', 2

In [9]:
# Print the top 15 words in each subreddit
for subreddit, top_words in top_dict_cv.items():
    print(subreddit)
    print(', '.join([word for word, count in top_words[0:30]]))
    print('---')

wsb
just, im, like, calls, dont, buy, tsla, today, going, money, day, puts, market, corn, good, bought, lol, spy, fuck, think, sell, shit, stock, time, got, people, fucking, right, tesla, thats
---
science
people, just, like, dont, im, think, know, thats, time, really, make, way, good, need, years, work, doesnt, youre, going, things, lot, use, isnt, does, say, study, right, better, actually, want
---
ama
im, like, did, just, dont, think, people, time, thats, happy, birthday, ive, know, want, good, say, lol, really, money, day, years, thanks, question, youre, lot, got, feel, thing, make, hope
---
askreddit
just, like, im, dont, people, time, know, think, good, really, years, got, didnt, ive, make, thats, want, did, life, way, sex, day, feel, right, going, youre, things, say, better, said
---
worldnews
people, just, like, dont, im, think, thats, russia, country, china, know, world, time, mask, going, right, really, youre, government, nuclear, way, good, new, make, vaccine, war, doesnt, w

In [21]:
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [20, 20]



for index, subreddit in enumerate(dtm_cv.columns):
    wc.generate(data_clean.text[subreddit])
    
    plt.subplot(4, 3, index + 1)
    plt.imshow(wc, interpolation='bilinear')
    plt.axis('off')
    plt.title(subreddit)

plt.show()

TypeError: unhashable type: 'csr_matrix'